# Beams - structural beams

This notebook shows how to use Beam elements model cantilever beams, how to get the results and how those results compare to theory.

A "Beam" in DAVE consists of a number of discrete segments and has a different formulation than the beams typically found in FEM packages.

In [ ]:
from DAVE import *
s = Scene();

Attempting to load:
 /home/ruben/miniconda3/lib/python3.8/site-packages/pyo3d.cpython-38-x86_64-linux-gnu.so
failed because:
dynamic module does not define module export function (PyInit_pyo3d)
If problems persist then removing this file from your system may help
The required version of pyo3d is not found on your system. No problem, we can download and install it automatically for you, proceed?


Beams are elastic nodes that can be created between axis systems.

In [ ]:
left = s.new_axis("left")
right = s.new_axis("right")
beam = s.new_beam("beam", nodeA=left, nodeB = right,
                        EA = 1000,
                        EIy = 1e5, EIz = 1e5, GIp = 1e3,
                        L=10,
                        mass = 0,
                        n_segments=20)
right.set_free()
s.solve_statics()

In [ ]:
from DAVE.jupyter import *;

In [ ]:
show(s, camera_pos=(5,-10,6), lookat = (5,0,2), sea=False, width=1000, height = 200)

A Beam node is rigidly fixed to the axis systems at its ends. It is fixed in the positive X-direction, so it departs from node A along the X-axis and arrives at ndoe B from the negative X-direction.

At the momement we have a massless beam with its left side fixed (axis system "left" is fixed) and its right size free (the right end of the beam is fixed to axis system "right", but "right" is not connected to anything and is free to move).

This is one of those cases that frequently occur in textbooks and for which we can easily check the results.

![cantilever.png](cantilever.png)

First place a point on the right side so that we can apply a force:

In [ ]:
s.new_point("point", parent = right)
force = s.new_force("force", parent = "point")

### Cantilever beam with a force

In [ ]:
force.force = (0,0,-100)

In [ ]:
s.solve_statics()

In [ ]:
show(s, camera_pos=(5,-10,0), lookat = (5,0,0), sea=False, width=1000, height = 300)

Theory tells us that the deflection and rotation of the right end should be:

$\delta = -PL^3 / 3EI$

and

$\Theta = PL^2 / 2EI$

Get the values from the model:

In [ ]:
L = beam.L
EI = beam.EIy
P = -force.force[2]

And calculate their values

The deflection is the vertical displacement of the axis system on the right side of the beam:

In [ ]:
right.z

In [ ]:
-P*L**3 / (3*EI)

Theta is the slope. The slope can be calculated from the rotation of the axis system, but it is also available as "tilt", which is a percentage.

In [ ]:
right.tilt_y / 100

In [ ]:
P*L**2 / (2*EI)

It is also possible to obtain the forces in the beam.

The moments are available at the nodes.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(beam.X_nodes, beam.bending[:,1])
plt.xlabel('Distance along the beam [m]')
plt.ylabel('Moment about y-axis [kN*m]');

## Cantilever with moment

We can do the same with a moment

In [ ]:
force.force = (0,0,0)
force.moment = (0,1000,0)

In [ ]:
s.solve_statics()

In [ ]:
show(s, camera_pos=(5,-10,3), lookat = (5,0,-1), sea=False, width=1000, height = 300)

Theory tells us that the deflection and rotation of the right end should be:

$\delta = ML^2 / 2EI$

and

$\Theta = ML / EI$

Get the values from the model:

In [ ]:
M = force.moment[1]  # moment about Y-axis

In [ ]:
M*L**2 / (2*EI)

In [ ]:
-right.z

In [ ]:
M*L / EI

In [ ]:
right.tilt_y / 100

## Tension and torsion

Beam can also take tension and torsion.

In [ ]:
force.force = (100,0,0)
force.moment = (100,0,0)

In [ ]:
s.solve_statics()

In [ ]:
show(s, camera_pos=(5,-10,3), lookat = (5,0,-1), sea=False, width=1000, height = 300)

Tension and torsion forces in the beam are available not at the nodes but at the centers for the beam segments.

In [ ]:
plt.plot(beam.X_midpoints, beam.torsion);

In [ ]:
plt.plot(beam.X_midpoints, beam.tension);

**Now this looks horrible!** 

Fortunately this is just the way matplotlib displays data which is almost constant.

Looking at the data directly it appears to be comfortingly constant:

In [ ]:
beam.torsion

In [ ]:
import matplotlib
matplotlib.rcParams['axes.formatter.useoffset'] = False

In [ ]:
plt.plot(beam.X_midpoints, beam.torsion);
plt.ylim((99.9, 100.1));

In [ ]:
plt.plot(beam.X_midpoints, beam.tension);
plt.ylim((99.9, 100.1));

## Distributed load

Modelling a distributed load is not possible in this way. Loads can only be added at points.

To model a distributed load we would have to manually discretise the model and add discrete loads at each node.

But these is a shortcut that we can take: The beams can have a mass. We can use the mass to model a distributed load as gravity.

In [ ]:
s.delete('force')

In [ ]:
q = 10 # kN/m

In [ ]:
beam.mass = beam.L * q  / 9.81

In [ ]:
s.solve_statics()

In [ ]:
show(s, camera_pos=(5,-10,0), lookat = (5,0,0), sea=False, width=1000, height = 300)

Theory tells us that the deflection and rotation of the right end should be:

$\delta = qL^4 / 8EI$

and

$\Theta = qL^3 / 6EI$

In [ ]:
-q*L**4 / (8*EI)

In [ ]:
right.z

In [ ]:
q*L**3 / (6*EI)

In [ ]:
right.tilt_y / 100

## Large deflections

All the tests done so far were performed on small deflections. That is for a good reason. The Euler/Bernouilli beam theory is only applicable to small displacements. This is because in this theory the moment is derived from the deflection:

$d^2/dx^2(EI d^2w / dx^2) = q$

so the moment in the beam is proprtional to the change in slope. This is only valid for small changes.

The implementation in DAVE **is valid for large deflections**, it is just that the analytical formula used above are not.

## Number of segments

Beams use a discrete model (see theory https://davedocs.online/beams.html).

When a higher number of segments is used:
- The model become more accurate
- The solver becomes slower 
- (The numerical errors build up)

So the number of nodes should be selected with some caution. In general 20 seems to be a good number for beams although in some cases (for example pure tension or torsion) a single segment is just as good.

It is easy to use the "plot_effect" function in scene to check the effect of the number of segments.

Here we calculate the effect of the number of segments on the vertical position of the second end of the beam:

In [ ]:
s.plot_effect(evaluate="s['point'].gz",
   change_property="n_segments",
    change_node="beam",
    start=1,
    to=25.0,
    steps=25);

-- end --